In [1]:
#%load_ext autoreload
#%autoreload 2
%reload_ext autoreload

from mfcc_pre_process import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical


# Second dimension of the feature is dim2
feature_dim_2 = 1378

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 10

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

Using TensorFlow backend.
Saving vectors of label - 'American': 100%|██████████| 68/68 [00:02<00:00, 26.73it/s]


In [2]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

In [3]:
X_train.shape, X_test.shape, y_train_hot.shape, y_test_hot.shape

((351, 20, 1378, 1), (235, 20, 1378, 1), (351, 10), (235, 10))

In [4]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 351 samples, validate on 235 samples
Epoch 1/50
351/351 [==============================] - 88s 252ms/step - loss: 3.3036 - acc: 0.1026 - val_loss: 2.3263 - val_acc: 0.0979
Epoch 2/50
351/351 [==============================] - 81s 230ms/step - loss: 2.3095 - acc: 0.1083 - val_loss: 2.3088 - val_acc: 0.1149
Epoch 3/50
351/351 [==============================] - 80s 228ms/step - loss: 2.2521 - acc: 0.1681 - val_loss: 2.2937 - val_acc: 0.1021
Epoch 4/50
351/351 [==============================] - 80s 229ms/step - loss: 2.2258 - acc: 0.1966 - val_loss: 2.2645 - val_acc: 0.1617
Epoch 5/50
351/351 [==============================] - 91s 259ms/step - loss: 2.2138 - acc: 0.1823 - val_loss: 2.2667 - val_acc: 0.1532
Epoch 6/50
351/351 [==============================] - 113s 322ms/step - loss: 2.2045 - acc: 0.1852 - val_loss: 2.2673 - val_acc: 0.1234
Epoch 7/50
351/351 [==============================] - 94s 268ms/step - loss: 2.1898 - acc: 0.1567 - val_loss: 2.2215 - val_acc: 0.1957
Epoch 8/

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")